# Make 15 min frequency and impute


In [1]:
# import libraries
import pandas as pd
import numpy as np


In [2]:
#load in data
df = pd.read_pickle('1.collect_data/data_files/5min.pkl')

In [3]:
df.head()

PaidOccupancy  ParkingSpaceCount
SourceElementKey OccupancyDateTime                                    
1001             2012-01-03 09:00:00            0.0                7.0
                 2012-01-03 09:05:00            0.0                7.0
                 2012-01-03 09:10:00            0.4                7.0
                 2012-01-03 09:15:00            1.0                7.0
                 2012-01-03 09:20:00            1.0                7.0

In [4]:
# make a new column for percent of occupied spaces
#df['PercentOccupied'] = df['PaidOccupancy']/df['ParkingSpaceCount']
#df['PercentOccupied'] = df['PercentOccupied'].clip(None, 1)

In [5]:
df.head()

PaidOccupancy  ParkingSpaceCount
SourceElementKey OccupancyDateTime                                    
1001             2012-01-03 09:00:00            0.0                7.0
                 2012-01-03 09:05:00            0.0                7.0
                 2012-01-03 09:10:00            0.4                7.0
                 2012-01-03 09:15:00            1.0                7.0
                 2012-01-03 09:20:00            1.0                7.0

In [6]:

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [7]:
level_values = df.index.get_level_values
blocks = level_values(0).unique()[:1]
num_blocks = len(blocks)

In [21]:
all_block_dfs = []
# for i, block_ind in enumerate(blocks):
#     print('block %d out of %d blocks' % (i, num_blocks))
#     dfs = []
#     for year in range(2012,2020):
        #df.loc[1001].index.hour.max()
block_ind = blocks[0]
i = 0
year = 2012
current_year = df.loc[block_ind][df.loc[block_ind].index.year == year]

In [22]:
current_year

,PaidOccupancy,ParkingSpaceCount
OccupancyDateTime,,
2012-01-03 09:00:00,0.0,7.0
2012-01-03 09:05:00,0.0,7.0
2012-01-03 09:10:00,0.4,7.0
2012-01-03 09:15:00,1.0,7.0
2012-01-03 09:20:00,1.0,7.0
...,...,...
2012-12-31 14:35:00,0.0,5.0
2012-12-31 14:40:00,0.0,5.0
2012-12-31 14:45:00,0.0,5.0


In [23]:
np.sum(current_year.isna())

PaidOccupancy        0
ParkingSpaceCount    0
dtype: int64

In [24]:
# level_values = df.index.get_level_values
# blocks = level_values(0).unique()[:1]
# num_blocks = len(blocks)

# all_block_dfs = []
# for i, block_ind in enumerate(blocks):
#     print('block %d out of %d blocks' % (i, num_blocks))
#     dfs = []
#     for year in range(2012,2020):
#         #df.loc[1001].index.hour.max()
#         current_year = df.loc[block_ind][df.loc[block_ind].index.year == year]
#         if len(current_year) > 0:
        
#             max_hour = current_year.index.hour.max()
#             min_hour = current_year.index.hour.min()

#             current_year = current_year.resample(rule='15min').mean()

#             # remove non paying hours:
#             current_year = current_year.iloc[current_year.index.indexer_between_time('%d:00:00' % min_hour, 
#                                                                   '%d:00:00' % max_hour, 
#                                                                   include_start=True, 
#                                                                   include_end=True)]
#             # remove sundays
#             current_year = current_year[current_year.index.dayofweek != 6]


#             # remove holidays

#             cal = calendar()
#             holidays = cal.holidays(start=current_year.index[0], end=current_year.index[-1])

#             current_year = current_year[~current_year.index.normalize().isin(holidays)]

#             # replace infs with nan
#             current_year = current_year.replace(np.inf, np.nan)            
            
#             #ffill parking spaces on block
#             current_year['ParkingSpaceCount'] = current_year['ParkingSpaceCount'].fillna(method='ffill')

#             # Fill nans with mean by time of day
#             mask = current_year.PaidOccupancy.isna()

#             filled_by_mean = current_year.groupby([current_year.index.hour]).transform(lambda x: x.fillna(x.mean()))
            
#             # this will show the values that were replaced
#             #filled_by_mean[mask]
            

#             dfs.append(filled_by_mean)

#     block_dfs = pd.concat(dfs).assign(SourceElementKey=block_ind).set_index('SourceElementKey', append=True).swaplevel(0,1)
#     all_block_dfs.append(block_dfs)


In [25]:
max_hour = current_year.index.hour.max()
min_hour = current_year.index.hour.min()

current_year = current_year.resample(rule='15min').mean()

# remove non paying hours:
current_year = current_year.iloc[current_year.index.indexer_between_time('%d:00:00' % min_hour, 
                                                      '%d:00:00' % max_hour, 
                                                      include_start=True, 
                                                      include_end=True)]
# remove sundays
current_year = current_year[current_year.index.dayofweek != 6]

In [26]:
# remove holidays

cal = calendar()
holidays = cal.holidays(start=current_year.index[0], end=current_year.index[-1])

current_year = current_year[~current_year.index.normalize().isin(holidays)]
#print(len(current_year), np.sum(current_year.isna()))
print(len(current_year))

current_year.head()

11198


,PaidOccupancy,ParkingSpaceCount
OccupancyDateTime,,
2012-01-03 09:00:00,0.133333,7.0
2012-01-03 09:15:00,1.000000,7.0
2012-01-03 09:30:00,1.000000,7.0
2012-01-03 09:45:00,1.800000,7.0
2012-01-03 10:00:00,1.933333,7.0


In [27]:
current_year['ParkingSpaceCount'] = current_year['ParkingSpaceCount'].fillna(method='ffill')

In [28]:
mask = current_year.PaidOccupancy.isna()

In [29]:
print(np.sum(mask))

4333


In [30]:
print(current_year[mask])

                     PaidOccupancy  ParkingSpaceCount
OccupancyDateTime                                    
2012-01-03 15:00:00            NaN                7.0
2012-01-03 15:15:00            NaN                7.0
2012-01-03 15:30:00            NaN                7.0
2012-01-03 15:45:00            NaN                7.0
2012-01-03 16:00:00            NaN                7.0
...                            ...                ...
2012-12-28 17:00:00            NaN                5.0
2012-12-31 08:00:00            NaN                5.0
2012-12-31 08:15:00            NaN                5.0
2012-12-31 08:30:00            NaN                5.0
2012-12-31 08:45:00            NaN                5.0

[4333 rows x 2 columns]


In [31]:
filled_by_mean = current_year.groupby([current_year.index.hour, current_year.index.minute]).transform(lambda x: x.fillna(x.mean()))
filled_by_mean


,PaidOccupancy,ParkingSpaceCount
OccupancyDateTime,,
2012-01-03 09:00:00,0.133333,7.0
2012-01-03 09:15:00,1.000000,7.0
2012-01-03 09:30:00,1.000000,7.0
2012-01-03 09:45:00,1.800000,7.0
2012-01-03 10:00:00,1.933333,7.0
...,...,...
2012-12-31 13:45:00,0.000000,5.0
2012-12-31 14:00:00,0.000000,5.0
2012-12-31 14:15:00,0.000000,5.0


In [32]:
filled_by_mean[mask]

,PaidOccupancy,ParkingSpaceCount
OccupancyDateTime,,
2012-01-03 15:00:00,1.339103,7.0
2012-01-03 15:15:00,1.451163,7.0
2012-01-03 15:30:00,1.364341,7.0
2012-01-03 15:45:00,1.285271,7.0
2012-01-03 16:00:00,1.162791,7.0
...,...,...
2012-12-28 17:00:00,1.000000,5.0
2012-12-31 08:00:00,0.071429,5.0
2012-12-31 08:15:00,0.077778,5.0


In [33]:
all_dfs = pd.concat(all_block_dfs)

ValueError: No objects to concatenate

In [ ]:
all_dfs['PercentOccupied'] = all_dfs['PaidOccupancy']/all_dfs['ParkingSpaceCount']

In [ ]:
all_dfs.to_pickle('1.collect_data/data_files/15min.pkl')